<h1>Vorhersage, ob es bei einem Unfall Schwerverletzte/Tote gibt</h1>

## 1. Vorbereitung des Datensatzes

### 1.1 Generelles Aufbereiten der Datenbasis

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
crash_data = gpd.read_file('../01_Source/U_VM_A_L_BRW_TL.geojson') 
geo_information = pd.read_csv('../01_Source/geo_information.csv', sep=';')
crash_data = pd.merge(crash_data, geo_information, on ='OBJECTID',  how='inner')

from bs4 import BeautifulSoup
import requests
import numpy as np
import datetime
crash_data2 = gpd.read_file('../01_Source/U_VM_A_L_BRW_TL.geojson') 

In [2]:
crash_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 13649 entries, 0 to 13648
Data columns (total 67 columns):
OBJECTID              13649 non-null int64
ULAND                 13649 non-null int64
UREGBEZ               13649 non-null int64
UKREIS                13649 non-null int64
UGEMEINDE             13649 non-null int64
UJAHR                 13649 non-null int64
UMONAT                13649 non-null int64
USTUNDE               13649 non-null int64
UWOCHENTAG            13649 non-null int64
UKATEGORIE            13649 non-null int64
UART                  13649 non-null int64
UTYP1                 13649 non-null int64
ULICHTVERH            13649 non-null int64
IstRad                13649 non-null int64
IstPKW                13649 non-null int64
IstFuss               13649 non-null int64
IstKrad               13649 non-null int64
IstGkfz               13649 non-null int64
IstSonstige           13649 non-null int64
STRZUSTAND            13649 non-null int64
LINREFX               1

In [3]:
crash_data["UKATEGORIE"] = crash_data["UKATEGORIE"]/2
crash_data['TARGET'] = 0 #0 leicht verletzt 1= Schwer Verletzt/getötet
crash_data.loc[crash_data["UKATEGORIE"] <= 1, ['TARGET']] = 1




# Unfallart
uart_df = pd.DataFrame({'UART' : [1,2,3,4,5,6,7,8,9,10], 'UART_Value':["Zusammenstoß mit anfahrendem / anhaltendem / ruhendem Fahrzeug",
                "Zusammenstoß mit vorausfahrendem / wartendem Fahrzeug",
                "Zusammenstoß mit seitlich in gleicher Richtung fahrendem Fahrzeug",
                "Zusammenstoß mit entgegenkommendem Fahrzeug",
                "Zusammenstoß mit einbiegendem / kreuzendem Fahrzeug",
                "Zusammenstoß zwischen Fahrzeug und Fußgänger",
                "Aufprall auf Fahrbahnhindernis",
                "Abkommen von Fahrbahn nach rechts",
                "Abkommen von Fahrbahn nach links",
                "Unfall anderer Art"]})

crash_data = pd.merge(crash_data, uart_df, on ='UART',  how='inner')

#Wochentag
wochentag_df = pd.DataFrame({'UWOCHENTAG':[1,2,3,4,5,6,7],
                                 'UWOCHENTAG_Value':['Sonntag','Montag','Dienstag','Mittwoch','Donnerstag','Freitag','Samstag']})
wochentag_df


crash_data = pd.merge(crash_data, wochentag_df, on ='UWOCHENTAG',  how='inner')

#Licht
licht_df = pd.DataFrame({'ULICHTVERH':[0,1,2],
                                 'ULICHTVERH_Value':['Tageslicht','Dämmerung','Dunkelheit']})


crash_data = pd.merge(crash_data, licht_df, on ='ULICHTVERH',  how='inner')
    
#Unfalltyp
unfalltyp_df = pd.DataFrame({'UTYP1':[1,2,3,4,5,6,7],
                                 'UTYP1_Value':['Fahrunfall','Abbiegeunfall','Einbiegen / Kreuzen-Unfall','Überschreitenunfall','Unfall durch ruhenden Verkehr','Unfall im Längsverkehr','sonstiger Unfall']})

crash_data = pd.merge(crash_data, unfalltyp_df, on ='UTYP1',  how='inner')

#Straßenzustand
str_zustand_df = pd.DataFrame({'STRZUSTAND':[0,1,2],
                                 'STRZUSTAND_Value':['trocken','nass/feucht/schlüpfrig','winterglatt']})
crash_data = pd.merge(crash_data, str_zustand_df, on ='STRZUSTAND',  how='inner')

crash_data['DTV'] = crash_data['DTV'].fillna(crash_data['DTV'].mean())
crash_data["ANTEIL_PKW"] = crash_data["PKW"]*100/crash_data["DTV"]
crash_data["ANTEIL_PKW"] = crash_data["ANTEIL_PKW"].fillna(crash_data["ANTEIL_PKW"].mean())
crash_data["ANTEIL_LKW"] = crash_data["LKW"]*100/crash_data["DTV"]
crash_data["ANTEIL_LKW"] = crash_data["ANTEIL_LKW"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data["ANTEIL_LIEFERWAGEN"] = crash_data["LIEFERWAGEN"]*100/crash_data["DTV"]
crash_data["ANTEIL_LIEFERWAGEN"] = crash_data["ANTEIL_LIEFERWAGEN"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data["ANTEIL_LINIENBUSSE"] = crash_data["LINIENBUSSE"]*100/crash_data["DTV"]
crash_data["ANTEIL_LINIENBUSSE"] = crash_data["ANTEIL_LINIENBUSSE"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['DTV']] = crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['LINIENBUSSE']]
crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['ANTEIL_LINIENBUSSE']] = 100
crash_data["ANTEIL_REISEBUSSE"] = crash_data["REISEBUSSE"]*100/crash_data["DTV"]
crash_data["ANTEIL_REISEBUSSE"] = crash_data["ANTEIL_REISEBUSSE"].fillna(crash_data["ANTEIL_REISEBUSSE"].mean())
crash_data["ANTEIL_KRAD"] = crash_data["KRAD"]*100/crash_data["DTV"]
crash_data["ANTEIL_KRAD"] = crash_data["ANTEIL_KRAD"].fillna(crash_data["ANTEIL_KRAD"].mean())
crash_data = crash_data.drop(["PKW","LKW","LIEFERWAGEN","LINIENBUSSE","REISEBUSSE","KRAD"], axis=1)
crash_data = crash_data.drop(["UKATEGORIE"], axis=1)

In [4]:
avg_ukreis = crash_data.groupby('UKREIS')['OBJECTID'].count()
count_target_ukreis = crash_data.groupby('UKREIS')['TARGET'].sum()
join_ukreis = pd.merge(avg_ukreis, count_target_ukreis, on ='UKREIS',  how='inner')
join_ukreis['UKREIS_AVG_TARGET'] = join_ukreis['TARGET']*100 / join_ukreis['OBJECTID']

avg_ugemeinde = crash_data.groupby('UGEMEINDE')['OBJECTID'].count()
count_target_ugemeinde = crash_data.groupby('UGEMEINDE')['TARGET'].sum()
join_ugemeinde = pd.merge(avg_ugemeinde, count_target_ugemeinde, on ='UGEMEINDE',  how='inner')
join_ugemeinde['UGEMEINDE_AVG_TARGET'] = join_ugemeinde['TARGET']*100 / join_ugemeinde['OBJECTID']

join_ugemeinde = join_ugemeinde.drop(["OBJECTID","TARGET"], axis=1)
join_ukreis = join_ukreis.drop(["OBJECTID","TARGET"], axis=1)

avg_dtv = crash_data.groupby('STR_NAME')['DTV'].mean()
count_crashes_by_street = crash_data.groupby('STR_NAME')['OBJECTID'].count()
count_fatal_crashes_by_street = crash_data.groupby('STR_NAME')['TARGET'].sum()
join_dtv = pd.merge(avg_dtv, count_crashes_by_street, on ='STR_NAME',  how='inner')
join_dtv = pd.merge(join_dtv, count_fatal_crashes_by_street, on ='STR_NAME',  how='inner')
join_dtv['CRASH_BY_DTV'] = (join_dtv['OBJECTID']*10000000 / join_dtv['DTV'])
join_dtv['CRASH_BY_DTV'] = join_dtv['CRASH_BY_DTV'].round(4)
join_dtv['CRASH_BY_DTV'] = join_dtv['CRASH_BY_DTV'] * 10000
join_dtv['FATAL_CRASH_BY_DTV'] = join_dtv['TARGET']*10000000 / join_dtv['DTV']
join_dtv['FATAL_CRASH_BY_DTV'] = join_dtv['FATAL_CRASH_BY_DTV'].round(4)
join_dtv['FATAL_CRASH_BY_DTV'] = join_dtv['FATAL_CRASH_BY_DTV'] * 10000

join_dtv.loc[join_dtv['DTV'] == 0, 'CRASH_BY_DTV'] = 0
join_dtv.loc[join_dtv['DTV'] == 0, 'FATAL_CRASH_BY_DTV'] = 0
join_dtv = join_dtv.drop(["OBJECTID","TARGET","DTV"], axis=1)

crash_data = pd.merge(crash_data, join_ukreis, on ='UKREIS',  how='inner')
crash_data = pd.merge(crash_data, join_ugemeinde, on ='UGEMEINDE',  how='inner')
crash_data = pd.merge(crash_data, join_dtv, on ='STR_NAME',  how='inner')
crash_data['FATAL_CRASH_BY_DTV'] = crash_data['FATAL_CRASH_BY_DTV'].fillna(crash_data['FATAL_CRASH_BY_DTV'].mean())
crash_data['CRASH_BY_DTV'] = crash_data['CRASH_BY_DTV'].fillna(crash_data['CRASH_BY_DTV'].mean())



In [5]:
#DTV,GESLAERM,BRW, NUTZUNG,WERT_VES,
crash_data.loc[crash_data["WERT_VES"] > 100, ['WERT_VES']] = 50
crash_data['WERT_VES'] = crash_data['WERT_VES'].fillna(50)
crash_data['UART_Value'] = crash_data['UART_Value'].fillna("Unbekannt")
crash_data['GESLAERM'] = crash_data['GESLAERM'].fillna("mittel")
crash_data['DTV'] = crash_data['DTV'].fillna(crash_data['DTV'].mean())
crash_data['DTV'] = crash_data['DTV'].fillna(0)

In [6]:
crash_data.head()

,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UART,UTYP1,ULICHTVERH,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,STRZUSTAND,LINREFX,LINREFY,METER,STR_NAME,DTV,HERKUNFT,NAME,KNAME,PLR_NAME,EW2010,VBUS,VBUS_PERS,GESLAERM,GESLAERM_PERS,GEMEINDE,BEZIRK,ANWERT,BRW,NUTZUNG,GFZ,VERFAHRENSART,STICHTAG,BEITRAGSZUSTAND,LUMNUM,ELEM_NR,VRICHT_TXT,WERT_VES,ZEIT_T,TAG_T,DURCH_T,DANN_T,DAT_T,geometry,LAT,LON,formatted_address,address_components,postal_code,sublocality,street_number,route,TARGET,UART_Value,UWOCHENTAG_Value,ULICHTVERH_Value,UTYP1_Value,STRZUSTAND_Value,ANTEIL_PKW,ANTEIL_LKW,ANTEIL_LIEFERWAGEN,ANTEIL_LINIENBUSSE,ANTEIL_REISEBUSSE,ANTEIL_KRAD,UKREIS_AVG_TARGET,UGEMEINDE_AVG_TARGET,CRASH_BY_DTV,FATAL_CRASH_BY_DTV
0,201103,11,0,10,10,2018,5,14,5,5,3,0,0,1,0,1,0,0,0,814741.4342,5.829904e+06,254.0,Lemkestr.,3852.0,"Modellrechnung 2014, VMZ",None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62550028_62550029.02,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814751.434 5829903.997, 814750...",52.528103,13.640801,"Lemkestr./Greifswalder Str. (Berlin), 12623 Be...","[{""long_name"": ""Lemkestr./Greifswalder Str. (B...",12623.0,Bezirk Marzahn-Hellersdorf,NaN,Lemkestr./Greifswalder Str. (Berlin),1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.92029,21.92029,121761153.0,48704461.0
1,199882,11,0,10,10,2018,4,16,6,5,3,0,1,1,0,0,0,0,0,814743.6764,5.829892e+06,254.0,Lemkestr.,3852.0,"Modellrechnung 2014, VMZ",None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62550028_62550029.02,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814753.676 5829892.369, 814753...",52.527997,13.640823,"Lemkestraße 104, 12623 Berlin, Germany","[{""long_name"": ""104"", ""short_name"": ""104"", ""ty...",12623.0,Bezirk Marzahn-Hellersdorf,104,Lemkestraße,1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Freitag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.92029,21.92029,121761153.0,48704461.0
2,204338,11,0,10,10,2018,8,7,6,5,3,0,0,1,0,0,0,0,0,814018.8594,5.828964e+06,67.0,Lemkestr.,2868.0,"Modellrechnung 2014, VMZ",None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],61540047_62540021.01,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814028.859 5828964.065, 814028...",52.520097,13.629297,"Albrecht-Dürer-Straße 5C, 12623 Berlin, Germany","[{""long_name"": ""5C"", ""short_name"": ""5C"", ""type...",12623.0,Bezirk Marzahn-Hellersdorf,5C,Albrecht-Dürer-Straße,0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Freitag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,80.613668,2.894003,10.216179,4.184100,0.557880,1.534170,21.92029,21.92029,121761153.0,48704461.0
3,196865,11,0,10,10,2018,1,18,2,5,3,2,0,1,0,0,0,0,0,814875.2039,5.830058e+06,220.0,Lemkestr.,6108.0,"Modellrechnung 2014, VMZ",None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62550001_62550021.01,beide Richtungen,30.0,07:00 - 17:00,Montag bis Freitag,Kinder,Zeitangabe,None,"MULTIPOLYGON (((814885.204 5830057.780, 814884...",52.529404,13.642912,"Lemkestraße 112B, 12623 Berlin, Germany","[{""long_name"": ""112B"", ""short_name"": ""112B"", ""...",12623.0,Bezirk Marzahn-Hel

In [7]:
crash_data = crash_data.drop(["ULAND","UREGBEZ","UJAHR","LINREFX","LINREFY","METER","EW2010","GFZ","HERKUNFT","NAME","KNAME","PLR_NAME",
                              "VBUS","VBUS_PERS","GESLAERM_PERS","GEMEINDE","BEZIRK","ANWERT","VERFAHRENSART","STICHTAG","BEITRAGSZUSTAND","LUMNUM","ELEM_NR","VRICHT_TXT","ZEIT_T","TAG_T"
                             ,"DURCH_T","DANN_T","DAT_T","LAT","LON","formatted_address","address_components","street_number","route","postal_code","sublocality","BRW","UART","UWOCHENTAG","ULICHTVERH","UTYP1","STRZUSTAND"], axis=1)
crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)

In [8]:
#np.all(np.isfinite(crash_data_copy))
#crash_data_copy = crash_data_copy.reset_index()
#np.where(crash_data_copy.values >= np.finfo(np.float64).max)

In [9]:
crash_data.head()

,UKREIS,UGEMEINDE,UMONAT,USTUNDE,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,DTV,GESLAERM,NUTZUNG,WERT_VES,geometry,TARGET,UART_Value,UWOCHENTAG_Value,ULICHTVERH_Value,UTYP1_Value,STRZUSTAND_Value,ANTEIL_PKW,ANTEIL_LKW,ANTEIL_LIEFERWAGEN,ANTEIL_LINIENBUSSE,ANTEIL_REISEBUSSE,ANTEIL_KRAD,UKREIS_AVG_TARGET,UGEMEINDE_AVG_TARGET,CRASH_BY_DTV,FATAL_CRASH_BY_DTV
0,10,10,5,14,0,1,0,1,0,0,3852.0,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814751.434 5829903.997, 814750...",1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.92029,21.92029,121761153.0,48704461.0
1,10,10,4,16,1,1,0,0,0,0,3852.0,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814753.676 5829892.369, 814753...",1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Freitag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.92029,21.92029,121761153.0,48704461.0
2,10,10,8,7,0,1,0,0,0,0,2868.0,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814028.859 5828964.065, 814028...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Freitag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,80.613668,2.894003,10.216179,4.184100,0.557880,1.534170,21.92029,21.92029,121761153.0,48704461.0
3,10,10,1,18,0,1,0,0,0,0,6108.0,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814885.204 5830057.780, 814884...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Montag,Dunkelheit,Einbiegen / Kreuzen-Unfall,trocken,81.777996,3.601834,10.216110,2.619515,0.261952,1.522593,21.92029,21.92029,121761153.0,48704461.0
4,10,10,5,20,0,1,0,0,0,0,3852.0,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814765.428 5829893.128, 814764...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Samstag,Dämmerung,Einbiegen / Kreuzen-Unfall,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.92029,21.92029,121761153.0,48704461.0


In [10]:
#crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)


crash_data_copy = crash_data.copy()
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UART_Value"], prefix='UART_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UWOCHENTAG_Value"], prefix='UWOCHENTAG_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UTYP1_Value"], prefix='UTYP1_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["ULICHTVERH_Value"], prefix='ULICHTVERH_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["STRZUSTAND_Value"], prefix='STRZUSTAND_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["NUTZUNG"], prefix='NUTZUNG_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["GESLAERM"], prefix='GESLAERM_')

#corr_matrix = crash_data_copy.corr()
#corr_matrix["TARGET"]
crash_data_copy = crash_data_copy.drop(["geometry"], axis=1)

In [11]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X = crash_data_copy.drop(["TARGET"] , axis=1)
Y = crash_data_copy["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [12]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier()

param_grid = {
                 'n_estimators': [5, 15, 30, 90, 100, 200],
                 'max_depth': [1, 2,3, 5, 7, 9, 11, 13],
                 'class_weight':["balanced"],
                 'min_samples_leaf': [2, 3, 4, 5, 7, 9]
             }

param_grid2 = {
                 'n_estimators': [85, 90, 95],
                 'max_depth': [3, 4, 5],
                 'min_samples_leaf': [7] #5
             }

from sklearn.model_selection import GridSearchCV

grid_clf = GridSearchCV(clf, param_grid, cv=10, verbose=10, n_jobs=5, scoring='balanced_accuracy')
grid_clf.fit(X_train, y_train)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(X_train,y_train)

y_pred=grid_clf.predict(X_test)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    1.8s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    1.9s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.9s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    2.1s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    2.4s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    2.7s
[Parallel(n_jobs=5)]: Done  51 tasks      | elapsed:    3.7s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:    4.2s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1966s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done  75 tasks      | elapsed:    4.3s
[Parallel(n_jobs=5)]: Done  94 tasks      | elapsed:    5.1s
[Parallel(n_jobs=5)]: Done 124 tasks      | elapsed:    7.1s
[Parallel(n_jobs=5)]: Done 154 tasks      | elapsed:    8.1s
[Parallel(n_jobs=5)]: Done 188 tasks      | elapsed:   10.0s
[Parallel(n_jobs=5)]: Done 222 tasks      | elapse

In [13]:
y_pred=grid_clf.predict(X_test)

In [14]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7037754114230397


In [15]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.26


In [16]:
grid_clf.best_params_

{'class_weight': 'balanced',
 'max_depth': 7,
 'min_samples_leaf': 2,
 'n_estimators': 200}

In [17]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1231, 474, 138, 223)

In [28]:
clf=RandomForestClassifier(random_state=42,class_weight='balanced',max_depth=7, min_samples_leaf=2, n_estimators=200)

clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [29]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7018393030009681


In [30]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1222, 483, 133, 228)

In [31]:
#Precision
tp/(tp+fp)

0.3206751054852321

In [32]:
#Recall
tp/(tp+fn)

0.631578947368421

In [23]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.26


In [24]:
feature_imp = pd.Series(clf.feature_importances_, index= X_train.columns).sort_values(ascending=False)

In [25]:
feature_imp

FATAL_CRASH_BY_DTV                                                               0.267747
UART_Value__Zusammenstoß mit vorausfahrendem / wartendem Fahrzeug                0.091536
UTYP1_Value__Überschreitenunfall                                                 0.052950
UTYP1_Value__Unfall im Längsverkehr                                              0.041177
IstKrad                                                                          0.039163
CRASH_BY_DTV                                                                     0.035587
UART_Value__Zusammenstoß zwischen Fahrzeug und Fußgänger                         0.031562
ANTEIL_KRAD                                                                      0.031056
UGEMEINDE_AVG_TARGET                                                             0.026948
ANTEIL_REISEBUSSE                                                                0.026104
UKREIS_AVG_TARGET                                                                0.025472
ANTEIL_LKW

In [26]:
crash_data_copy["TARGET"].value_counts()

0    8586
1    1743
Name: TARGET, dtype: int64

In [27]:
(11433-2216)/11433

0.806175107145981